In [1]:
import pandas, numpy

In [2]:
import scipy, scipy.signal, scipy.optimize
import statsmodels, statsmodels.stats, statsmodels.stats.multitest

In [3]:
import matplotlib, matplotlib.pyplot
matplotlib.rcParams.update({'font.size':20, 
                            'font.family':'sans-serif', 
                            'xtick.labelsize':16, 
                            'ytick.labelsize':16, 
                            'figure.figsize':(16*(2/3), 9*(2/3)), 
                            'axes.labelsize':20
                           })

In [4]:
input_file = '230117_Arnar.lfq.proteins_From_Gudrun.xlsx'

In [5]:
df = pandas.read_excel(input_file)

In [6]:
df.columns

Index(['Protein Group', 'Top', 'Accession', 'Significance', 'Coverage (%)',
       '#Peptides', '#Unique', 'PTM', 'ADRNOR_4H_1_1_2792 Area',
       'ADRNOR_4H_1_2_2793 Area', 'ADRNOR_4H_2_1_2795 Area',
       'ADRNOR_4H_2_2_2796 Area', 'ADRNOR_4H_3_1_2798 Area',
       'ADRNOR_4H_3_2_2799 Area', 'ADR_4H_1_1_2772 Area',
       'ADR_4H_1_2_2773 Area', 'ADR_4H_2_1_2775 Area', 'ADR_4H_2_2_2776 Area',
       'ADR_4H_3_1_2778 Area', 'ADR_4H_3_2_2779 Area', 'CTRL_4H_1_1_2752 Area',
       'CTRL_4H_1_2_2753 Area', 'CTRL_4H_2_1_2755 Area',
       'CTRL_4H_2_2_2756 Area', 'CTRL_4H_3_1_2758 Area',
       'CTRL_4H_3_2_2759 Area', 'NOR_4H_1_1_2781 Area', 'NOR_4H_1_2_2782 Area',
       'NOR_4H_2_1_2786 Area', 'NOR_4H_2_2_2787 Area', 'NOR_4H_3_1_2789 Area',
       'NOR_4H_3_2_2790 Area', 'TNF_4H_1_1_2761 Area', 'TNF_4H_1_2_2762 Area',
       'TNF_4H_2_1_2764 Area', 'TNF_4H_2_2_2765 Area', 'TNF_4H_3_1+_2769 Area',
       'TNF_4H_3_2_2770 Area', 'ADRNOR_24H_1_1_2931 Area',
       'ADRNOR_24H_1_2_2932 A

In [7]:
subset = []
for element in df.columns:
    if 'ADRNOR_4H_' in element:
        subset.append(element)
    if 'CTRL_4H_' in element:
        subset.append(element)

print(len(subset), subset)
useful_info = ['Accession'] + subset
wdf = df.loc[:, useful_info]
wdf.set_index('Accession', inplace=True)

print(wdf.columns)
labels01 = wdf.columns.values[[0, 1]]
labels02 = wdf.columns.values[[2, 3]]
labels03 = wdf.columns.values[[4, 5]]

labels04 = wdf.columns.values[[6, 7]]
labels05 = wdf.columns.values[[8, 9]]
labels06 = wdf.columns.values[[10, 11]]

wdf.insert(0, 'c3', wdf.loc[:, labels06].median(axis=1))
wdf.insert(0, 'c2', wdf.loc[:, labels05].median(axis=1))
wdf.insert(0, 'c1', wdf.loc[:, labels04].median(axis=1))

wdf.insert(0, 'a3', wdf.loc[:, labels03].median(axis=1))
wdf.insert(0, 'a2', wdf.loc[:, labels02].median(axis=1))
wdf.insert(0, 'a1', wdf.loc[:, labels01].median(axis=1))

wdf.head()

12 ['ADRNOR_4H_1_1_2792 Area', 'ADRNOR_4H_1_2_2793 Area', 'ADRNOR_4H_2_1_2795 Area', 'ADRNOR_4H_2_2_2796 Area', 'ADRNOR_4H_3_1_2798 Area', 'ADRNOR_4H_3_2_2799 Area', 'CTRL_4H_1_1_2752 Area', 'CTRL_4H_1_2_2753 Area', 'CTRL_4H_2_1_2755 Area', 'CTRL_4H_2_2_2756 Area', 'CTRL_4H_3_1_2758 Area', 'CTRL_4H_3_2_2759 Area']
Index(['ADRNOR_4H_1_1_2792 Area', 'ADRNOR_4H_1_2_2793 Area',
       'ADRNOR_4H_2_1_2795 Area', 'ADRNOR_4H_2_2_2796 Area',
       'ADRNOR_4H_3_1_2798 Area', 'ADRNOR_4H_3_2_2799 Area',
       'CTRL_4H_1_1_2752 Area', 'CTRL_4H_1_2_2753 Area',
       'CTRL_4H_2_1_2755 Area', 'CTRL_4H_2_2_2756 Area',
       'CTRL_4H_3_1_2758 Area', 'CTRL_4H_3_2_2759 Area'],
      dtype='object')


,a1,a2,a3,c1,c2,c3,ADRNOR_4H_1_1_2792 Area,ADRNOR_4H_1_2_2793 Area,ADRNOR_4H_2_1_2795 Area,ADRNOR_4H_2_2_2796 Area,ADRNOR_4H_3_1_2798 Area,ADRNOR_4H_3_2_2799 Area,CTRL_4H_1_1_2752 Area,CTRL_4H_1_2_2753 Area,CTRL_4H_2_1_2755 Area,CTRL_4H_2_2_2756 Area,CTRL_4H_3_1_2758 Area,CTRL_4H_3_2_2759 Area
Accession,,,,,,,,,,,,,,,,,,
Q01105|SET_HUMAN,65495.316500,85802.715000,107049.96800,74730.06000,58426.502000,79745.793000,66207.93000,64782.70300,86486.8400,85118.59000,109897.17000,104202.7660,76131.59000,73328.5300,52312.2270,64540.77700,81183.08600,78308.50000
P30520|PURA2_HUMAN,37783.377000,45287.551000,71625.20500,60713.17700,49429.075000,51736.767500,37504.72700,38062.02700,46083.1760,44491.92600,75813.75000,67436.6600,58889.98400,62536.3700,50495.8700,48362.28000,51730.57000,51742.96500
H3BSJ5|H3BSJ5_HUMAN,1248.864050,1276.184400,665.27760,1023.67045,1008.338200,586.370725,710.18670,1787.54140,1401.3933,1150.97550,738.94460,591.6106,994.67620,1052.6647,1322.4603,694.21610,583.13250,589.60895
Q92900|RENT1_HUMAN,54415.652000,58750.342500,9318.80320,54141.58050,35658.565000,56442.270000,50964.09400,57867.21000,57027.4900,60473.19500,5592.36040,13045.2460,61990.89500,46292.2660,33900.8500,37416.28000,55339.03000,57545.51000
B7ZBQ3|B7ZBQ3_HUMAN,343.877655,487.092185,919.51117,280.56381,314.254565,353.206300,328.46777,359.28754,479.5121,494.67227,956.61804,882.4043,300.37222,260.7554,288.9870,339.52213,371.89246,334.52014


In [8]:
columns2drop = wdf.columns[6:]
columns2drop
wdf.drop(columns=columns2drop, inplace=True)

In [9]:
treatment_median = wdf.loc[:, ['a1', 'a2', 'a3']].median(axis=1)
reference_median = wdf.loc[:, ['c1', 'c2', 'c3']].median(axis=1)
log2FC = treatment_median - reference_median

wdf.insert(0, 'a_median', treatment_median)
wdf.insert(0, 'c_median', reference_median)
wdf.insert(0, 'log2FC', log2FC)

wdf.head()

,log2FC,c_median,a_median,a1,a2,a3,c1,c2,c3
Accession,,,,,,,,,
Q01105|SET_HUMAN,11072.65500,74730.060000,85802.715000,65495.316500,85802.715000,107049.96800,74730.06000,58426.502000,79745.793000
P30520|PURA2_HUMAN,-6449.21650,51736.767500,45287.551000,37783.377000,45287.551000,71625.20500,60713.17700,49429.075000,51736.767500
H3BSJ5|H3BSJ5_HUMAN,240.52585,1008.338200,1248.864050,1248.864050,1276.184400,665.27760,1023.67045,1008.338200,586.370725
Q92900|RENT1_HUMAN,274.07150,54141.580500,54415.652000,54415.652000,58750.342500,9318.80320,54141.58050,35658.565000,56442.270000
B7ZBQ3|B7ZBQ3_HUMAN,172.83762,314.254565,487.092185,343.877655,487.092185,919.51117,280.56381,314.254565,353.206300


In [10]:
# drop a few cases where the protein is the same for both conditions. Avoid testing, it gives nans and screws multiple correction
print(wdf.shape)
elements2drop = wdf[wdf['c_median'] == wdf['a_median']].index
print(elements2drop)
wdf.drop(index=elements2drop, inplace=True)
print(wdf.shape)

(7841, 9)
Index(['Q7Z406|MYH14_HUMAN', 'H0YC82|H0YC82_HUMAN',
       'A0A7I2V3I0|A0A7I2V3I0_HUMAN', 'H3BUX2|H3BUX2_HUMAN',
       'Q9UI12|VATH_HUMAN', 'P00846|ATP6_HUMAN', 'O60828|PQBP1_HUMAN',
       'Q96FJ2|DYL2_HUMAN', 'Q01814|AT2B2_HUMAN', 'Q96A29|FUCT1_HUMAN',
       'Q9HD34|LYRM4_HUMAN', 'P60228|EIF3E_HUMAN'],
      dtype='object', name='Accession')
(7829, 9)


In [11]:
result = scipy.stats.ttest_ind(wdf.loc[:, ['c1', 'c2', 'c3']], wdf.loc[:, ['a1', 'a2', 'a3']], axis=1)
wdf.insert(0, 'Student P', result.pvalue)

result = scipy.stats.mannwhitneyu(wdf.loc[:, ['c1', 'c2', 'c3']], wdf.loc[:, ['a1', 'a2', 'a3']], axis=1)
wdf.insert(0, 'Mann P', result.pvalue)

wdf.head()

,Mann P,Student P,log2FC,c_median,a_median,a1,a2,a3,c1,c2,c3
Accession,,,,,,,,,,,
Q01105|SET_HUMAN,0.382733,0.328188,11072.65500,74730.060000,85802.715000,65495.316500,85802.715000,107049.96800,74730.06000,58426.502000,79745.793000
P30520|PURA2_HUMAN,0.662521,0.835754,-6449.21650,51736.767500,45287.551000,37783.377000,45287.551000,71625.20500,60713.17700,49429.075000,51736.767500
H3BSJ5|H3BSJ5_HUMAN,0.382733,0.480619,240.52585,1008.338200,1248.864050,1248.864050,1276.184400,665.27760,1023.67045,1008.338200,586.370725
Q92900|RENT1_HUMAN,1.000000,0.667692,274.07150,54141.580500,54415.652000,54415.652000,58750.342500,9318.80320,54141.58050,35658.565000,56442.270000
B7ZBQ3|B7ZBQ3_HUMAN,0.190430,0.199642,172.83762,314.254565,487.092185,343.877655,487.092185,919.51117,280.56381,314.254565,353.206300


In [12]:
print(wdf[wdf['Student P'] < 0.05].shape)
print(wdf[wdf['Mann P'] < 0.05].shape)

print(numpy.min(wdf['Student P']))
print(numpy.min(wdf['Mann P']), numpy.max(wdf['Mann P']))

(619, 11)
(0, 11)
3.338571928200455e-05
0.07652250047505924 1.0


In [13]:
corrected = statsmodels.stats.multitest.fdrcorrection(wdf['Student P'].values)
print(numpy.min(corrected[1]), numpy.max(corrected[1]))

corrected = statsmodels.stats.multitest.multipletests(wdf['Student P'].values, method='fdr_bh')
print(numpy.min(corrected[1]))

wdf.insert(0, 'Student adjuted P', corrected[1])

0.20449171721508613 0.9999563112532082
0.20449171721508613


In [14]:
wdf['Student adjuted P'].sort_values(ascending=True)

Accession
Q9NX62|IMPA3_HUMAN             0.204492
Q9Y6B6|SAR1B_HUMAN             0.204492
Q14533|KRT81_HUMAN             0.213935
O00592|PODXL_HUMAN             0.213935
P07332|FES_HUMAN               0.213935
                                 ...   
E9PFN4|E9PFN4_HUMAN            0.999956
Q9NWS0|PIHD1_HUMAN             0.999956
A0A0C4DGB0|A0A0C4DGB0_HUMAN    0.999956
P46087|NOP2_HUMAN              0.999956
A6NGW1|A6NGW1_HUMAN            0.999956
Name: Student adjuted P, Length: 7829, dtype: float64

In [15]:
asd

NameError: name 'asd' is not defined

In [ ]:
intensity = numpy.log2(reference.values + 1)
print(numpy.min(intensity), numpy.max(intensity), intensity)

hist, bin_edges = numpy.histogram(intensity, bins=100, range=(0, 20))
print(sum(hist), len(intensity))
half_bin = (bin_edges[1] - bin_edges[0])/2
x = bin_edges + half_bin
x = x[:-1]




#yhat = scipy.signal.savgol_filter(hist, 51, 3)


def func(x, a, x0, sigma): 
    return a*numpy.exp(-(x-x0)**2/(2*sigma**2)) 

popt, pcov  = scipy.optimize.curve_fit(func, x, hist)
yhat = func(x, popt[0], popt[1], popt[2]) 
cdf = numpy.cumsum(yhat)/numpy.sum(yhat)
print(cdf)

matplotlib.pyplot.plot(x, hist, 'o', color='skyblue', alpha=1/2, mec='none')
matplotlib.pyplot.plot(x, yhat, '-', lw=2, alpha=1/2)

matplotlib.pyplot.grid(ls=':', alpha=1/2)
matplotlib.pyplot.show()


In [ ]:
def gauss(x, H, A, x0, sigma):
    return H + A * numpy.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def gauss_fit(x, y):
    mean = sum(x * y) / sum(y)
    sigma = numpy.sqrt(sum(y * (x - mean) ** 2) / sum(y))
    popt, pcov = scipy.optimize.curve_fit(gauss, x, y, p0=[min(y), max(y), mean, sigma])
    return popt

xdata = x
ydata = hist

H, A, x0, sigma = gauss_fit(xdata, ydata)
matplotlib.pyplot.plot(x, hist, 'o', color='skyblue', alpha=1/2)
matplotlib.pyplot.plot(x, gauss(xdata, *gauss_fit(xdata, ydata)), '--r', label='fit')